Versionado de datos con DVC --- 0:00 min
===

* 0:00 min | Ultima modificación: Abril 4, 2022 | YouTube

In [1]:
#
# Limpieza del área de trabajo
#
!rm -rf /tmp/dvcdemo /tmp/dvcstore

Instalación
---

La instalación se realiza usando `pip`:

```bash
pip install dvc
```

Creación de una carpeta temporal para el demo
---

In [2]:
!mkdir /tmp/dvcdemo
%cd /tmp/dvcdemo

/tmp/dvcdemo


Inicializacion de git
---

In [3]:
!git init
!git config user.email jdvelasq@unal.edu.co
!git config user.name jdvelasq

Initialized empty Git repository in /tmp/dvcdemo/.git/


Inicialización de DVC
--

In [4]:
!dvc init

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


Chequeo de la inicialización
--

In [5]:
!git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   .dvc/.gitignore
	new file:   .dvc/config
	new file:   .dvcignore



In [6]:
#
# Note que en la inicialización se agregaron los archivos
#
#  .dvc/.gitignore
#  .dvc/config
#  .dvcignore
#
# pero seran ignorados por ahora.
#
!ls -al

total 20
drwxr-xr-x 4 root root 4096 Apr 22 19:27 .
drwxrwxrwt 1 root root 4096 Apr 22 19:27 ..
drwxr-xr-x 3 root root 4096 Apr 22 19:27 .dvc
-rw-r--r-- 1 root root  139 Apr 22 19:27 .dvcignore
drwxr-xr-x 7 root root 4096 Apr 22 19:27 .git


Configuración del remoto para almacenar las versiones de los datos
--

In [7]:
#
# Opciones del comando remote
#
!dvc remote --help

usage: dvc remote [-h] [-q | -v] {add,default,modify,list,remove,rename} ...

Set up and manage data remotes.
Documentation: <https://man.dvc.org/remote>

positional arguments:
  {add,default,modify,list,remove,rename}
                        Use `dvc remote CMD --help` for command-specific help.
    add                 Add a new data remote.
    default             Set/unset the default data remote.
    modify              Modify the configuration of a data remote.
    list                List all available data remotes.
    remove              Remove a data remote.
    rename              Rename a DVC remote

optional arguments:
  -h, --help            show this help message and exit
  -q, --quiet           Be quiet.
  -v, --verbose         Be verbose.


In [8]:
#
# Almacenamiento en una carpeta local
#
!dvc remote add --default myremote /tmp/dvcstore

Setting 'myremote' as a default remote.


In [9]:
#
# Listado de los remotos existentes
#
!dvc remote list

myremote	/tmp/dvcstore


In [10]:
#
# Se almacena la nueva configuración
#
!git commit .dvc/config -m "Configure local remote in /tmp/dvcstore"
!git tag -a "init" -m "Init version"

[master (root-commit) bc12dae] Configure local remote in /tmp/dvcstore
 1 file changed, 4 insertions(+)
 create mode 100644 .dvc/config


Descarga de datos de ejemplo con DVC
---

In [11]:
#
# Creación de la carpeta de datos dentro de la carpeta actual
#
!mkdir -p data
!ls -l 

total 4
drwxr-xr-x 2 root root 4096 Apr 22 19:27 data


In [12]:
#
# Descarga los datos desde el repositorio de ejemplo de dvc
#
repo = "https://github.com/iterative/dataset-registry"
src = "get-started/data.xml"
dst = "data/data.xml"

!dvc get {repo} {src} -o {dst}

In [13]:
#
# Contenido de la carpeta de datos
#
!ls -lh data

total 37M
-rw-r--r-- 1 root root 37M Apr 22 19:27 data.xml


Seguimiento de cambios en los datos
---

In [14]:
!dvc add --help

usage: dvc add [-h] [-q | -v] [-R] [--no-commit] [--external] [--glob]
               [--file <filename>] [-o <path>] [--to-remote] [-r <name>]
               [-j <number>] [--desc <text>]
               targets [targets ...]

Track data files or directories with DVC.
Documentation: <https://man.dvc.org/add>

positional arguments:
  targets               Input files/directories to add.

optional arguments:
  -h, --help            show this help message and exit
  -q, --quiet           Be quiet.
  -v, --verbose         Be verbose.
  -R, --recursive       Recursively add files under directory targets.
  --no-commit           Don't put files/directories into cache.
  --external            Allow targets that are outside of the DVC repository.
  --glob                Allows targets containing shell-style wildcards.
  --file <filename>     Specify name of the .dvc file this command will
                        generate.
  -o <path>, --out <path>
                        Destination path to pu

In [15]:
#
# Se inicia el tracking del archivo
#
!dvc add --quiet data/data.xml

⠙ Checking graphph


In [16]:
#
# Contenido de la carpeta
#
!ls -lh data/

total 37M
-rw-r--r-- 1 root root 37M Apr 22 19:27 data.xml
-rw-r--r-- 1 root root  80 Apr 22 19:27 data.xml.dvc


In [17]:
#
# Archivo de tracking
#
!cat data/data.xml.dvc

outs:
- md5: a304afb96060aad90176268345e10355
  size: 37891850
  path: data.xml


In [18]:
#
# Se adiciona el tracking de los cambios como tal.
#
!git add data/data.xml.dvc data/.gitignore
!git commit -m "Add raw data"
!git tag -a "v0" -m "raw data"

[master ffcbb9a] Add raw data
 4 files changed, 11 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvcignore
 create mode 100644 data/.gitignore
 create mode 100644 data/data.xml.dvc


Almacenamiento de datos en el remoto
---

In [19]:
#
# Se almacenan la versión actual de los datos en el remoto
#
!dvc push --quiet

Recuperación
---

In [20]:
#
# Se borran los datos existentes
#
!rm -rf .dvc/cache
!rm -f data/data.xml

#
# Se recuperan los datos
# 
!dvc pull --quiet

In [21]:
!ls -lh data/

total 37M
-rw-r--r-- 1 root root 37M Apr 22 19:28 data.xml
-rw-r--r-- 1 root root  80 Apr 22 19:27 data.xml.dvc


Cambios en los datos
---

In [22]:
#
# Simulación de más datos.
# Se duplican los datos originales.
#
!cp data/data.xml /tmp/data.xml
!cat /tmp/data.xml >> data/data.xml

In [23]:
!ls -lh data/

total 73M
-rw-r--r-- 1 root root 73M Apr 22 19:28 data.xml
-rw-r--r-- 1 root root  80 Apr 22 19:27 data.xml.dvc


In [24]:
#
# Se adicionan los datos al tracking
#
!dvc add --quiet data/data.xml

⠙ Checking graphph


In [25]:
!git commit data/data.xml.dvc -m "augmented data"
!git tag -a "v1" -m "augmented data"

[master 1a19a09] augmented data
 1 file changed, 2 insertions(+), 2 deletions(-)


In [26]:
!dvc push --quiet

Cambios entre versiones de los datos
---

In [27]:
#
# Versión anterior de los datos
#
!git checkout v0 data/data.xml.dvc

Updated 1 path from ac492b1


In [28]:
!ls -lh data/

total 73M
-rw-r--r-- 1 root root 73M Apr 22 19:28 data.xml
-rw-r--r-- 1 root root  80 Apr 22 19:28 data.xml.dvc


In [29]:
!git commit data/data.xml.dvc -m "Revert dataset updates"

[master 76e0c4b] Revert dataset updates
 1 file changed, 2 insertions(+), 2 deletions(-)


In [30]:
# !dvc remote remove myremote 
# !dvc remote remove /temp/databackup